## CESM2 - LARGE ENSEMBLE (LENS2)

- This Notebook aims to compute the MOC upper branch and compare the results with the observations. The upper branch is the integrated volume transport from the surface to the level where the meridional volume transport inverts.
- In addition, we intend to obtain time series of the maximum value of the MOC for all latitudes so as to analyze its variability. 

### Imports

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np  
import dask
import cf_xarray
import intake
import cftime
import nc_time_axis
import intake_esm
import matplotlib.pyplot as plt
import pop_tools
from dask.distributed import Client
from ncar_jobqueue import NCARCluster
import warnings, getpass, os
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from scipy.integrate import cumtrapz
from scipy import integrate
from mpl_toolkits.axes_grid1 import make_axes_locatable

### Dask

In [ ]:
mem_per_worker = 30 # memory per worker in GB 
num_workers = 80 # number of workers
cluster = NCARCluster(cores=1, processes=3, memory=f'{mem_per_worker} GB',resource_spec=f'select=1:ncpus=6:mem={mem_per_worker}GB', walltime='8:00:00')
cluster.scale(num_workers)
client = Client(cluster)
print(client)
client

### Read in OGCM history file & MOC template file

In [ ]:
catalog = intake.open_esm_datastore(
    '/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cesm2-le.json'
)

In [ ]:
cat_subset = catalog.search(component='ocn',variable=['MOC'],frequency='month_1')

In [ ]:
# Load catalog entries for subset into a dictionary of xarray datasets
dset_dict_raw  = cat_subset.to_dataset_dict(zarr_kwargs={'consolidated': True}, storage_options={'anon': True})
print(f'\nDataset dictionary keys:\n {dset_dict_raw.keys()}')

In [ ]:
# Variables
fb=(['MOC'])
pd=('historical','ssp370')
ff=('cmip6','smbb')
for ifb in range(0,len(fb)):
    for iff in range(0,len(ff)):
        for ipd in range(0,len(pd)):
            str=f'ds_{pd[ipd]}_{ff[iff]}_{fb[ifb]} = dset_dict_raw[\'ocn.{pd[ipd]}.pop.h.{ff[iff]}.{fb[ifb]}\']'
            exec(str)
        str=f'ds_{ff[iff]}_{fb[ifb]}=xr.combine_nested([ds_{pd[0]}_{ff[iff]}_{fb[ifb]},ds_{pd[1]}_{ff[iff]}_{fb[ifb]}],concat_dim=[\'time\']);'
        exec(str)
    str=f'ds_{fb[ifb]}=xr.combine_nested([ds_{ff[0]}_{fb[ifb]},ds_{ff[1]}_{fb[ifb]}],concat_dim=[\'member_id\']);'
    exec(str) 
print(f'Done!')

### Vertical profile

In [ ]:
ila=-34.5
alpha=0.05
moc_mean=ds_MOC.MOC.isel(transport_reg=1).sel(lat_aux_grid=ila, method='nearest').sum(dim='moc_comp').mean(dim='time')
moc_mean.coords['moc_z']=moc_mean.coords['moc_z']*-0.01
moc_mean.plot.line(y='moc_z',color='black',alpha=alpha,linewidth=1,add_legend=False)
moc_mean.mean(dim='member_id').plot.line(y='moc_z',color='black',linewidth=1,label='LENS2')

plt.xlabel("Volume Transport [Sv]")
plt.ylabel("Depth [m]")
plt.grid(color='k', linestyle='-', linewidth=0.4)
bbox_props = dict(boxstyle='round', fc='w', ec='0.5', alpha=0.9)
plt.xlim(-2.5, 20)
plt.legend()
plt.show()

### Interpolate to create AMOC data in the middle of the layer
- First, let's interpolate to a latitude to evaluate the interpolation method. Then we will interpolate to all latitudes

In [ ]:
prof = np.empty((len(ds_MOC['moc_z'])-1)) * np.nan
for t in range(0,len(ds_MOC.coords['moc_z'])-1):
        prof[t]=(ds_MOC.coords['moc_z'][t]+ds_MOC.coords['moc_z'][t+1])/2 
ds_MOC.MOC.interp(moc_z=prof,method='cubic').isel(member_id=0,time=0,transport_reg=1).sel(lat_aux_grid=ila, method='nearest').sum(dim='moc_comp').plot()
ds_MOC.MOC.isel(member_id=0,time=0,transport_reg=1).sel(lat_aux_grid=ila, method='nearest').sum(dim='moc_comp').plot()
ds_MOC_int=ds_MOC.isel(transport_reg=1).sel(lat_aux_grid=ila, method='nearest').sum(dim='moc_comp').interp(moc_z=prof,method='cubic')

- Now, let's interpolate for all as latitudes

In [ ]:
prof = np.empty((len(ds_MOC['moc_z'])-1)) * np.nan
for t in range(0,len(ds_MOC.coords['moc_z'])-1):
        prof[t]=(ds_MOC.coords['moc_z'][t]+ds_MOC.coords['moc_z'][t+1])/2 
ds_MOC_int=ds_MOC.isel(transport_reg=1).sum(dim='moc_comp').interp(moc_z=prof,method='cubic')

### AMOC upper branch
- Let's calculate the upper branch of the AMOC, defined as the integral of the volume transport up to the depth at which the AMOC reverses sign. Since there can be a reversal of this velocity at the surface, let's bound the buffers by this reversal so that it is below 1000m. Since our goal is to compare AMOC from LENS2 with other ensembles, we will bound the latitude interval between the equator and 40S and the time from 1990 to 2022. 

In [ ]:
ds_MOC_int=ds_MOC_int.MOC.sel(time=slice('1993-01-01','2021-12-31')).sel(lat_aux_grid=[-20,-25,-30,-34.5], method='nearest')

In [ ]:
dz=ds_MOC.dz

In [ ]:
%%time
dp1=40 # first depth
dp2=59 # second depth
y_int = np.empty((len(ds_MOC_int['member_id']),len(ds_MOC_int['time']),len(ds_MOC_int['lat_aux_grid']))) * np.nan
for it in range(0,len(ds_MOC_int['time'])): # time 
    print(f'Time: {it}')
    amoc = ds_MOC_int.isel(time=it)
    for im in list(range(0,100,10)):
    #for im in range(0,len(ds_MOC_int['member_id'])):
        for il in range(0,len(ds_MOC_int['lat_aux_grid'])):
            L = np.where(amoc.isel(member_id=im,lat_aux_grid=il,moc_z=slice(dp1,dp2))<0) # Notes: dp1<z<dp2
            if len(amoc['moc_z'][L])==0: # If there is no negative transport in that interval, the integral is done from surface to bottom.     
                y_int[im,it,il]=sum(amoc.isel(member_id=im,lat_aux_grid=il)*dz.isel(
                    member_id=im,time=it).values)/sum(dz.isel(
                    member_id=im,time=it).values)
            else:
                y_int[im,it,il]=sum(amoc.isel(member_id=im,lat_aux_grid=il,moc_z=slice(0,L[0][0]-1+dp1))*dz.isel(
                    member_id=im,time=it,z_t=slice(0,L[0][0]-1+dp1)).values)/sum(dz.isel(
                    member_id=im,time=it,z_t=slice(0,L[0][0]-1+dp1)).values)

In [ ]:
ds_MOC_SAMOC = xr.Dataset({'amoc_upper': (('member_id','time','lat_aux_grid'), y_int)},coords={'member_id': ds_MOC_int.coords['member_id'],'time': ds_MOC_int.coords['time'],'lat_aux_grid': ds_MOC_int.coords['lat_aux_grid']})
ds_MOC_SAMOC.attrs['description'] = 'Meridional Overturning Circulation (MOC) upper branch from the surface to the level where the transport volume reverses sign for the period from 1990 to 2022'
ds_MOC_SAMOC.attrs['units'] = 'Sv'
ds_MOC_SAMOC.attrs['author'] = 'Mauricio Rocha'
ds_MOC_SAMOC.attrs['email'] = 'mauricio.rocha@usp.br'
ds_MOC_SAMOC['time'] = ds_MOC_SAMOC['time'].astype('datetime64[ns]')